In [ ]:
import os
import sys
os.chdir('../')
sys.path.append('../')

In [12]:
import torch

a = torch.rand((2,2,3))
b = torch.rand((6))

b.shape+(1,) #concat(torch.tnesor(([2])))

torch.Size([6, 1])

In [9]:
import torch
import torch.nn.functional as F
a = torch.rand((1,1,2,2,3))
b = torch.tensor([[1,1,1],[2,2,2]]).float().view(1,2,1,3,1)
c = torch.matmul(a,b)
a,b,c,c.shape

(tensor([[[[[0.2930, 0.2536, 0.8373],
            [0.2090, 0.0838, 0.3000]],
 
           [[0.0630, 0.4348, 0.0460],
            [0.8475, 0.5611, 0.3318]]]]]),
 tensor([[[[[1.],
            [1.],
            [1.]]],
 
 
          [[[2.],
            [2.],
            [2.]]]]]),
 tensor([[[[[1.3840],
            [0.5929]],
 
           [[0.5438],
            [1.7405]]],
 
 
          [[[2.7680],
            [1.1858]],
 
           [[1.0875],
            [3.4810]]]]]),
 torch.Size([1, 2, 2, 2, 1]))

In [2]:
import torch
a = torch.rand((2,1,5,5))
b = torch.zeros((2,3,5,5))
b[:,:,:,:] = a
a, b

(tensor([[[[0.2399, 0.5392, 0.4271, 0.9112, 0.7183],
           [0.3900, 0.7354, 0.5409, 0.6545, 0.7653],
           [0.5342, 0.4016, 0.7155, 0.0555, 0.4240],
           [0.5617, 0.5221, 0.3007, 0.2620, 0.9398],
           [0.5106, 0.2386, 0.6631, 0.9306, 0.5173]]],
 
 
         [[[0.0166, 0.6163, 0.3757, 0.2405, 0.7030],
           [0.8404, 0.9824, 0.9697, 0.5186, 0.7443],
           [0.8495, 0.6359, 0.6595, 0.3478, 0.9964],
           [0.8697, 0.7699, 0.1160, 0.6130, 0.3718],
           [0.1238, 0.3422, 0.3206, 0.9934, 0.3864]]]]),
 tensor([[[[0.2399, 0.5392, 0.4271, 0.9112, 0.7183],
           [0.3900, 0.7354, 0.5409, 0.6545, 0.7653],
           [0.5342, 0.4016, 0.7155, 0.0555, 0.4240],
           [0.5617, 0.5221, 0.3007, 0.2620, 0.9398],
           [0.5106, 0.2386, 0.6631, 0.9306, 0.5173]],
 
          [[0.2399, 0.5392, 0.4271, 0.9112, 0.7183],
           [0.3900, 0.7354, 0.5409, 0.6545, 0.7653],
           [0.5342, 0.4016, 0.7155, 0.0555, 0.4240],
           [0.5617, 0.5221, 0.300

In [ ]:
b, torch.sum(torch.log(b),dim=1)


In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('data/test')

In [ ]:
import torch
writer.add_embedding(torch.rand((5,5)),meta)

In [15]:
import torch
r = torch.zeros((2,2,3,4)).long()
a = torch.rand((2,2,3))
b = torch.ones((2,3,1))
r[:,:,:,:3] = a.unsqueeze(dim=2)
r[:,:,:,3:] = b.unsqueeze(dim=1)
r[:,:,:,2] = 3
r

tensor([[[[0, 0, 3, 1],
          [0, 0, 3, 1],
          [0, 0, 3, 1]],

         [[0, 0, 3, 1],
          [0, 0, 3, 1],
          [0, 0, 3, 1]]],


        [[[0, 0, 3, 1],
          [0, 0, 3, 1],
          [0, 0, 3, 1]],

         [[0, 0, 3, 1],
          [0, 0, 3, 1],
          [0, 0, 3, 1]]]])

In [16]:
a = torch.rand((1,3))
a.expand((3,1,3))

tensor([[[0.3344, 0.4434, 0.3447]],

        [[0.3344, 0.4434, 0.3447]],

        [[0.3344, 0.4434, 0.3447]]])